In [155]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

import collections
import json

In [34]:
df = pd.read_excel("data/buzzsumo_fakenews.xlsx")
df.loc[df["Source"] == "Lapada Lapada – Informações em Tempo Real", "Source"] = "Lapada Lapada"
df.loc[df["Source"].str.contains("Uol"), "Source"] = "UOL"
df.loc[df["Source"].str.contains("UOL"), "Source"] = "UOL"
df.loc[df["Source"].str.contains("uol"), "Source"] = "UOL"

In [35]:
df_uol = df.loc[df["Source"] == "UOL"]

In [36]:
df_uol.loc[df_uol["URL"].str.contains("bol.uol.com.br"), "Source"] = "BOL"
df_uol.loc[df_uol["URL"].str.contains("noticias.uol.com.br"), "Source"] = "UOL"
df_uol.loc[df_uol["URL"].str.contains("7segundos.com.br"), "Source"] = "7 Segundos"
df_uol.loc[df_uol["URL"].str.contains("jc.ne10.uol.com.br"), "Source"] = "Jornal do Commercio"
df_uol.loc[df_uol["URL"].str.contains("noticiasdatv.uol.com.br"), "Source"] = "Notícias da TV"
df_uol.loc[df_uol["URL"].str.contains("operamundi.uol.com.br"), "Source"] = "Opera Mundi"
df_uol.loc[df_uol["URL"].str.contains("paranaportal.uol.com.br"), "Source"] = "Paraná Portal"
df_uol.loc[df_uol["URL"].str.contains("radiojornal.ne10.uol.com.br"), "Source"] = "Rádio Jornal"
df_uol.loc[df_uol["URL"].str.contains("redetv.uol.com.br"), "Source"] = "RedeTV!"
df_uol.loc[df_uol["URL"].str.contains("cultura.uol.com.br"), "Source"] = "TV Cultura"

In [40]:
df_uol = df_uol.loc[df_uol["Source"] == "UOL"]

In [47]:
df_uol = df_uol.reset_index()

In [164]:
def get_newsbody_uol(soup):
    newsbody = soup.find("div", "text")
    newsbody.find("div", "slot-m").extract()
    return newsbody.get_text(" ")

In [165]:
def get_metadata_uol(soup):
    text = soup.find("script", type="application/ld+json").get_text()
    return json.loads(text)[0]

In [170]:
news_body_list = []
headline_list = []
description_list = []
author_list = []
for _, row in df_uol.iterrows():
    response = requests.get(row["URL"])
    if response.ok:
        soup = BeautifulSoup(response.text, "html.parser")
        try:
            news_body_list.append(get_newsbody_uol(soup))
        except Exception as e:
            news_body_list.append(e)
        try:
            metadata = get_metadata_uol(soup)
        except Exception as e:
            metadata = {"headline" : e}
        
        headline_list.append(metadata.get("headline"))
        description_list.append(metadata.get("description"))
        author_list.append(metadata.get("author").get("name") if metadata.get("author") else None)
    else:
        news_body_list.append(response.status_code)
        headline_list.append(None)
        description_list.append(None)
        author_list.append(None)

    
df_uol["News Body"] = news_body_list
df_uol["Headline"] = headline_list
df_uol["Description"] = description_list
df_uol["Author"] = author_list

In [172]:
df_uol[["URL", "Source", "News Body", "Headline", "Description", "Author"]].to_excel("Notícias UOL.xlsx", index=False)